# `spaxelsleuth` examples (S7)
---
In this notebook, you will learn how to use `spaxelsleuth` to 
1. create a `pandas` DataFrame containing spaxel-by-spaxel information for galaxies in the S7 survey;
2. use the included plotting tools to analyse the S7 data;
3. create plots that you can use to analyse specific galaxies. 

In [ ]:
from spaxelsleuth import load_user_config
load_user_config("/home/u5708159/.spaxelsleuthconfig.json")
from spaxelsleuth.config import settings

## Create the DataFrames
---
### Creating the "metadata" DataFrame
After you have installed `spaxelsleuth` and have downloaded the necessary files from [DataCentral](http://datacentral.org.au) (see the README), the first step is to create the "metadata" DataFrame containing basic information about each S7 galaxy, such as redshifts and stellar masses.

In [ ]:
from spaxelsleuth.io.io import make_metadata_df
make_metadata_df(survey="s7")

In [ ]:
from spaxelsleuth.io.io import load_metadata_df
df_metadata = load_metadata_df(survey="s7")

In [ ]:
df_metadata

### Creating the S7 DataFrame
The first step is to create the DataFrame containing all measured quantities, such as emission line fluxes, stellar and gas kinematics, and extinction, for each individual spaxel in the S7 galaxies. The resulting DataFrames are saved to file and can be accessed using `load_s7_df()`.

The input arguments are as follows:
* `gals` is a list of galaxies for which you want to create the DataFrame. If unspecified, it will run on the full S7 sample.
* `eline_SNR_min` gives the minimum S/N for which a fitted emission line component will be accepted. All quantities associated with emission line components that fall below this S/N threshold (e.g. kinematics, fluxes, etc.) will be set to `np.nan`.
* `correct_extinction` controls whether extinction correction is applied to the emission line fluxes or not.
* `metallicity_diagnostics` controls which strong-line metallicity diagnostics are evaluated.

In [ ]:
from spaxelsleuth.io.io import make_df

In [ ]:
make_df(survey="s7",
        bin_type="default",
        ncomponents="merge",
        eline_SNR_min=3,
        eline_ANR_min=3,
        correct_extinction=True,
        metallicity_diagnostics=["N2Ha_PP04",],
        nthreads=15)


### Load the S7 DataFrame we just created

In [ ]:
from spaxelsleuth.io.io import load_df

df = load_df(
    survey="s7",
    ncomponents="merge",
    bin_type="default",
    correct_extinction=True,
    eline_SNR_min=3,
    eline_ANR_min=3,
)

In addition to the measured quantities released in DR3, the DataFrame also contains various other measurements, such as line ratios, spectral categories, equivalent widths, and the like:

In [ ]:
df

In [ ]:
for col in df.columns:
    print(col)

In [ ]:
# Calculate the number of spaxels with different spectral classifications 
n_spaxels_tot = df.shape[0]
spec_cats = df["BPT (total)"].unique()  # List of spectral categories in the data set
for spec_cat in spec_cats:
    cond = df["BPT (total)"] == spec_cat
    n_spaxels = df[cond].shape[0]
    print(f"There are {n_spaxels} spaxels ({n_spaxels / n_spaxels_tot * 100:.2f}%) classified as {spec_cat}")

In [ ]:
# Histograms showing the distribution in velocity dispersion
import matplotlib.pyplot as plt
from astropy.visualization import hist
fig, ax = plt.subplots(nrows=1, ncols=1)
hist(df[f"sigma_gas (component 1)"].values, bins="scott", ax=ax, range=(0, 500), density=False, histtype="step")
ax.set_xlabel(r"$\sigma_{\rm gas}$")
ax.set_ylabel(r"$N$")

You can also select a subset of the full data sample as follows:

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1)
for cat in df["BPT (total)"].unique():
    df_cat = df[df["BPT (total)"] == cat]
    hist(df_cat[f"sigma_gas (component 1)"].values, bins="scott", ax=ax, range=(0, 500), density=False, histtype="step", label=cat)
ax.set_xlabel(r"$\sigma_{\rm gas}$")
ax.set_ylabel(r"$N$")
ax.legend()

## Using `spaxelsleuth.plotting`
--- 
In the below cells, we will demonstrate how to use various functions in `spaxelsleuth.plotting` to make different kinds of plots useful for analysing the data set.

In [ ]:
from spaxelsleuth.plotting.plottools import plot_empty_BPT_diagram, plot_BPT_lines
from spaxelsleuth.plotting.plotgalaxies import plot2dhistcontours, plot2dscatter

#### Plot the distribution of spaxels in the WHAN diagram
The WHAN diagram, developed by Cid Fernandes et al. (2010, 2011), plots the H$\alpha$ equivalent width (EW) against the [NII]6583Å/H$\alpha$ line ratio, and can be used to distinguish between emission by star formation, AGN and hot, low-mass evolved stars.

In [ ]:
# Plot a 2D histogram showing the distribution of S7 spaxels in the WHAN diagram
_ = plot2dhistcontours(df=df,
              col_x=f"log N2 (total)",
              col_y=f"log HALPHA EW (total)",
              col_z="count", log_z=True,
              plot_colorbar=True)

In [ ]:
# Repeat the above, but colour by the median BPT classification
_ = plot2dhistcontours(df=df,
              col_x=f"log N2 (total)",
              col_y=f"log HALPHA EW (total)",
              col_z="BPT (total)", 
              plot_colorbar=True)

In [ ]:
# Repeat the above, but colour by the D4000Å break strength
_ = plot2dhistcontours(df=df,
              col_x=f"log N2 (total)",
              col_y=f"log HALPHA EW (total)",
              col_z="D4000", 
              plot_colorbar=True)

In [ ]:
# If we like, we can use a different colourmap using the "cmap" keyword.
# The contour colours can also be changed using the "colors" keyword, and the levels can be changed using "levels".
# The axis limits can also be changed using the "xmin", "xmax", "ymin", "ymax" and "vmin", "vmax" keyword.
# We can also change the number of histogram bins using the "nbins" keyword.
import numpy as np
_ = plot2dhistcontours(df=df,
              col_x=f"log N2 (total)", xmin=-0.6, xmax=0.0,
              col_y=f"log HALPHA EW (total)", 
              col_z="D4000", vmin=1.1, vmax=2.0,
              nbins=50, 
              cmap="cubehelix", colors="yellow", levels=np.logspace(0, 4, 5),
              plot_colorbar=True)

#### Plot optical diagnostic ("BPT") diagrams for the full sample, showing the distribution of spaxels in the S7 galaxies

In [ ]:
# Plot an empty BPT diagram
fig, axs_bpt = plot_empty_BPT_diagram(nrows=1)

# Plot 2D histograms showing the distribution of the entire sample
for cc, col_x in enumerate(["log N2", "log S2", "log O1"]):
    # Add BPT demarcation lines
    plot_BPT_lines(ax=axs_bpt[cc], col_x=col_x)    

    # Plot histograms showing distribution for whole sample
    _ = plot2dhistcontours(df=df,
                  col_x=f"{col_x} (total)",
                  col_y=f"log O3 (total)",
                  col_z="count", log_z=True,
                  ax=axs_bpt[cc],
                  cax=None,
                  plot_colorbar=True if cc==2 else False)

    if cc > 0:
        axs_bpt[cc].set_ylabel("")
    axs_bpt[cc].grid()

#### Repeat the above, but colour the histograms by $\sigma_{\rm gas}$

In [ ]:
# Plot an empty BPT diagram
fig, axs_bpt = plot_empty_BPT_diagram(nrows=1)

# Plot 2D histograms showing the distribution of the entire sample
for cc, col_x in enumerate(["log N2", "log S2", "log O1"]):
    # Add BPT demarcation lines
    plot_BPT_lines(ax=axs_bpt[cc], col_x=col_x)    

    # Plot histograms showing distribution for whole sample
    _ = plot2dhistcontours(df=df,
                  col_x=f"{col_x} (total)",
                  col_y=f"log O3 (total)",
                  col_z="sigma_gas (component 1)", 
                  vmax=150,  # Tweak the z-axis scaling to bring out details 
                  ax=axs_bpt[cc],
                  cax=None, colors="white",
                  plot_colorbar=True if cc==2 else False)
    
    if cc > 0:
        axs_bpt[cc].set_ylabel("")
    axs_bpt[cc].grid()

## Using `spaxelsleuth.plotting` to plot individual galaxies
--- 
In the below cells, we will demonstrate how create plots based on individual S7 galaxies using functions in `spaxelsleuth.plotting`.


In [ ]:
cond = df_metadata["ID (string)"] == "ESO202-G23"
gal = df_metadata.loc[cond, "ID"].values[0]

#### Plot 2D maps showing various quantities in this galaxy
Given a DataFrame corresponding to a single galaxy, `plot2dmap()` reconstructs a 2D image of a specified column (e.g., velocity dispersion) from the rows and plots it. 

In [ ]:
from spaxelsleuth.plotting.plot2dmap import plot2dmap
fig, ax = plot2dmap(df=df, gal=gal, col_z="BPT (total)")
fig, ax = plot2dmap(df=df, gal=gal, col_z="log N2 (total)")
fig, ax = plot2dmap(df=df, gal=gal, col_z="D4000")
fig, ax = plot2dmap(df=df, gal=gal, col_z="v_gas (component 1)")
fig, ax = plot2dmap(df=df, gal=gal, col_z="v_gas (component 2)")
fig, ax = plot2dmap(df=df, gal=gal, col_z="v_gas (component 3)")
fig, ax = plot2dmap(df=df, gal=gal, col_z="sigma_gas (component 1)")
fig, ax = plot2dmap(df=df, gal=gal, col_z="sigma_gas (component 2)")
fig, ax = plot2dmap(df=df, gal=gal, col_z="sigma_gas (component 3)")

In [ ]:
# Playing with input arguments 
# As in plot2dhistcontours, the maximum colour scaling can be set using "vmin" and "vmax" and the colourmap can be changed using "cmap". 
# The quantity that is used to plot contours can be set to any column in the DataFrame using "col_z_contours" and the colours and linewidths can be configured using "colours" and "linewidths".
_ = plot2dmap(df=df, gal=gal, col_z="v_gas (component 1)",
              cmap="jet", vmin=-50, vmax=+50,
              col_z_contours="v_gas (component 1)", levels=[-40, -20, 0, 20, 40], colors="pink", linewidths=2.5)


#### Scatter plots 
Create a BPT diagram with 2D histograms and contours showing the distribution in the full sample, with the data points from this galaxy overlaid on top using a scatter plot.

In [ ]:
# Plot an empty BPT diagram
fig, axs_bpt = plot_empty_BPT_diagram(nrows=1)

# Plot 2D histograms showing the distribution of the entire sample
for cc, col_x in enumerate(["log N2", "log S2", "log O1"]):
    # Add BPT demarcation lines
    plot_BPT_lines(ax=axs_bpt[cc], col_x=col_x)    

    # Plot histograms showing distribution for whole sample
    _ = plot2dhistcontours(df=df,
                  col_x=f"{col_x} (total)",
                  col_y=f"log O3 (total)",
                  col_z="count", log_z=True,
                  cmap="bone_r", alpha=0.4,
                  ax=axs_bpt[cc],
                  plot_colorbar=False)
    
    # Scatter plot for this galaxy
    _ = plot2dscatter(df=df, gal=gal, 
                      col_x=f"{col_x} (total)",
                      col_y=f"log O3 (total)",
                      col_z=f"v_gas (component 1)",
                      ax=axs_bpt[cc],
                      vmin=-100, vmax=+100,
                      plot_colorbar=True if cc==2 else False)
    
# Turn of y-axis labels 
[ax.set_ylabel("") for ax in axs_bpt[1:]]